In [12]:
import torch
from torch import nn
from torch.nn import functional as F

In [13]:
net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))

X = torch.rand(2, 20)

net(X)

tensor([[-0.2190, -0.1323,  0.2588, -0.2323, -0.1302, -0.0796, -0.1368,  0.0924,
          0.0519, -0.0995],
        [-0.1316, -0.0530,  0.2712, -0.2001,  0.0042, -0.1553, -0.0302,  0.0572,
          0.0716, -0.0524]], grad_fn=<AddmmBackward0>)

In [14]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn. Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

# 实例化多层感知机层数，然后在每次调用正向传播时调用这些层

In [15]:
net = MLP()
net(X)
# net.forward(X)

tensor([[-0.0948, -0.2147,  0.0622,  0.2143,  0.2623, -0.1060,  0.0349,  0.0429,
          0.0554,  0.1752],
        [-0.1100, -0.2716,  0.0495,  0.0974,  0.2539, -0.0507,  0.0097,  0.1172,
          0.0310,  0.2291]], grad_fn=<AddmmBackward0>)

# 实现sequential

In [16]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block # _modules  存储层的容器
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1677,  0.1681,  0.0553, -0.0756, -0.0843,  0.1097,  0.1357,  0.1228,
         -0.0381,  0.0110],
        [-0.2476,  0.0546,  0.1296,  0.0082, -0.1523,  0.0788,  0.1555, -0.0134,
         -0.0524,  0.0598]], grad_fn=<AddmmBackward0>)

# 在forward正向传播函数中执行代码

In [17]:
class FixHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixHiddenMLP()
net(X)

tensor(-0.1299, grad_fn=<SumBackward0>)

# 混合使用

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64),
                                nn.ReLU(),
                                nn.Linear(64, 32),
                                nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixHiddenMLP())
chimera(X)

tensor(-0.0362, grad_fn=<SumBackward0>)